## Step 1: Install all the required packages 

In [1]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
# pip install scipy

## Step 2: Import all the required libraries 

In [2]:
import os 
import torch 
from datasets import load_dataset 
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging 
from peft import LoraConfig, PeftModel 
from trl import SFTTrainer

/rhome/f111169109/.conda/envs/albygpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Incase of Llama-2, the following prompt template is used for the chat models 

System prompt (optinal) to guide the model 

User prompt (required) to give instruction 

Model Answer (required)

```
<s>
[INST] 

<<SYS>>
System prompt
<</SYS>>

User prompt 

[/INST] 

Model answer 

</s>
```

### Reformat instruction dataset to follow Llama-2 template

Orignal Dataset: https://huggingface.co/datasets/timdettmers/openassistant-guanaco

Reformat Dataset following the Llama 2 template with 1k sample: https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k

Complete Reformat Dataset following the Llama 2 template: https://huggingface.co/datasets/mlabonne/guanaco-llama2

To know how this dataset was created, you can check this notebook: https://colab.research.google.com/drive/1Ad7a9zMmkxuXTOh1Z7-rNSICA4dybpM2?usp=sharing

> Note: You don't need to follow a specific prompt template if you are using the base Llama-2 model instead of the chat version.

### How to fine tune Llama 2
- Free Goolge Colab offers a 15GB Graphics Card (Limited Resources --> Barely enough to store Llama 2-7b's weights).
- We also need to consider the overhead due to optimier states, gradients, and forward activations.
- Full fine-tunning is not possible here: We need parameter-efficient fine-tunning (PEFT) techniques like LoRA or QLoRA.
- To drastically reduce the VRAM usage, we must fine-tune the model in 40bit precision, which is why we'll use QLoRA here.

## Step 3
1. Load a llama-2-7b-chat-hf model (chat model)
2. Train it on the mlabonne/guanaco-llama-2-1k (1000 samples). Which will produce our fine-tuned model Llama-2-7b-chat-finetune 

QloRA will use a rank of 64 with a scaling parameter of 16. We'll load the Llama-2 model directly in 4-bit precision using the NF4 type and train it for one epoch.

In [3]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

In [4]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [5]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [6]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

In [7]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

## Step 4: Load everything and start the fine-tuning process 
1. First of all, we want to load the dataset we defined. Here, our dataset is already preprocessed but, usually, this is where you would reformat the prompt. Filter our bad text, combine multiple datasets, etc. 
2. Then, we are configuring bitsandbytes for 4-bit quantization.
3. Next, we are loading the Llama-2 model in 4-bit precision one GPU with the coresspoding tokenizer. 
4. Finally, we are loading configuration for QLoRA, regular training parameters, and passing everything to the SFTrainer. The training can finally start!

In [8]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

In [9]:
# Load tokenizer and model with QLoRA configuration 
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit, 
    bnb_4bit_quant_type=bnb_4bit_quant_type, 
    bnb_4bit_compute_dtype=compute_dtype, 
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [11]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [12]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/rhome/f111169109/.conda/envs/albygpt/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.06it/s]


In [13]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [14]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [15]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [16]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

/rhome/f111169109/.conda/envs/albygpt/lib/python3.9/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/rhome/f111169109/.conda/envs/albygpt/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/rhome/f111169109/.conda/envs/albygpt/lib/python3.9/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
You're using a LlamaTokenizerFast tokenizer. Please note 

Step,Training Loss
25,1.572400
50,1.473000
75,1.313100
100,1.351500
125,1.399300
150,1.202700
175,1.267200
200,1.239900
225,1.313700
250,1.283500


TrainOutput(global_step=375, training_loss=1.300577173868815, metrics={'train_runtime': 2109.3632, 'train_samples_per_second': 1.422, 'train_steps_per_second': 0.178, 'total_flos': 2.684361754312704e+16, 'train_loss': 1.300577173868815, 'epoch': 3.0})

In [17]:
# Save trained model
trainer.model.save_pretrained(new_model)

## Step 5: Check the plots on tensorboard, as follows

In [18]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

## Step 6: Use the text generation pipeline to ask questions like "What is large language model?" Note that I'm formatting the input to match Llama-2 prompt template

In [19]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/rhome/f111169109/.conda/envs/albygpt/lib/python3.9/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/rhome/f111169109/.conda/envs/albygpt/lib/python3.9/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] What is a large language model? [/INST] A large language model is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate human-like language. everybody is talking about the large language models, but what are they?

Large language models are neural networks that are trained on large datasets of text to generate human-like language. They are trained on a large dataset of text, such as books, articles, and websites, and are designed to generate text that is similar to the text in the training dataset.

Large language models are trained using a technique called "masked language modeling." In this technique, the model is trained to predict the next word in a sequence of text, given the previous words in the sequence. The model is trained on a large dataset of text, and the goal is to generate text that is similar to the text in the training dataset.

Large language models are


In [20]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

## Step 7: Store new Llama-2 model (Llama-2-7b-chat-finetune)

How can we store our new Llama-2-7b-chat-finetune model --> We need to merge the weights from LoRA with the base model. Unfortunately, as far as I know, there is no straightforward way to do it: We nedd to reload the base model in FP16 precision and use the peft library to merge everything.

In [21]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.13s/it]


In [22]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"